<a href="https://colab.research.google.com/github/samanthasu12/TA-Strike-Analysis-For-Daily-Nexus/blob/main/OverallSentiment_TA_Strike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install praw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.8/188.8 KB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 KB 3.0 MB/s eta 0:00:00


In [2]:
#imports for reddit scraping/diagrams/sentiment analysis
import praw
import pandas as pd
import datetime
import nltk                                
import matplotlib.pyplot as plt            
import random                             
import re                                 
import string                                     
from nltk.stem import PorterStemmer       
from nltk.tokenize import sent_tokenize  
from textblob import TextBlob
import seaborn as sns
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize, RegexpTokenizer 
from nltk.corpus import stopwords
from pandas import option_context

# visualization
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (10, 8) 
import seaborn as sns
sns.set(style='whitegrid', palette='Dark2')
from wordcloud import WordCloud

In [ ]:
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 200

In [3]:
#my developer access code
reddit_read_only = praw.Reddit(client_id="RGdKiAFJPXF58ib0Tr4fcw",
                               client_secret="1hXSX11y-nMi8Fobsp9NDMiT7s0mMA",
                               user_agent="TA Strike Scraper UCSB",
                               check_for_async=False)
#the subreddit that I'm using
subreddit = reddit_read_only.subreddit("UCSantaBarbara")

#checking for keywords
keyword_list = ["strike", "graduate student workers" "salary", "wages", "cola", " uaw ", "union", "tentative agreement", "picket"]

In [4]:
#all posts

posts = subreddit.new(limit=4000)

posts_dict0 = {"Title": [], "Post Text": [], "Score": [], "ID": [], "Total Comments": [], "Total Number of Comments":[]}
 
for post in posts:
  dt=datetime.datetime.utcfromtimestamp(post.created_utc)
  if datetime.datetime(2022,11,7,0,0,0) < dt < datetime.datetime(2022,12,25,11,59,59):
    posts_dict0["Title"].append(post.title)
    posts_dict0["Post Text"].append(post.selftext)
    posts_dict0["Score"].append(post.score)
    posts_dict0["ID"].append(post.id)
    random_comment_list0=[]
    post.comments.replace_more(limit=0)
    for comment in post.comments.list():
      random_comment_list0.append(comment.body)
    posts_dict0["Total Comments"].append(random_comment_list0)
    posts_dict0["Total Number of Comments"].append(post.num_comments)

week0_posts = pd.DataFrame(posts_dict0)
week0_posts

,Title,Post Text,Score,ID,Total Comments,Total Number of Comments
0,Anyone staying in IV for Christmas?,"Hey everyone, since I’m staying in IV for Chri...",49,zuvxyp,[Damn bruh sorry for you but merry Christmas; ...,17
1,Dropped PSTAT 8 on GOLD and it also dropped my...,Merry Christmas. I was looking at GOLD because...,1,zutubg,[The system doesn’t just drop people. It’s eit...,4
2,160b with Shkolnik,Has anyone taken 160b with Shkolnik? I've hear...,2,zup0on,"[What department is this? PSTAT 160B?, Last wi...",12
3,Mental Illness Resources?,"Hello everyone. Recently, I’ve been thinking a...",32,zunvbu,"[Skip caps, if you want actual help schedule a...",7
4,Is Financial Math not a major anymore on GOLD?,So basically I am doing Grad School applicatio...,16,zuhqwz,"[No, GOLD doesn’t recognize your major anymore...",9
...,...,...,...,...,...,...
531,can you get prescribed for medication through ...,i haven’t used ucsb shs that much but i want t...,7,ywg18x,"[Yup! Just make an appointment, i went to shs ...",6
532,Passing grade for MCDB 101A,I was wondering if anyone who has taken this c...,4,ywf686,"[That midterm today sheeeesh, I heard the clas...",3
533,Internationals during thanksgiving week,What are you guys going to do? I’m worried abo...,13,ywd5cu,[i am going to play video games for 100 hours ...,11
534,What went on here last night?,,56,ywb04x,"[average minor in possession traffic stop 🙏😔, ...",13


In [5]:
#function for checking if something contains the keyword
def has_any_keyword(mystring,keywords):
  for word in mystring.split():
    for keyword in keywords:
      if keyword in word.lower():
        return True
  return False
has_any_keyword(r"Apparently it was up on Gradescope for like 20 minutes on Monday so it's done being graded and they are just withholding it for some reason, my friend was their score but I missed it before they removed it", keyword_list)

False

In [6]:
newdf0 = pd.DataFrame(posts_dict0)
list0=[]
for i in range(0,len(week0_posts)):
  if has_any_keyword(week0_posts['Title'].iloc[i],keyword_list):
    list0.append(i)
  if has_any_keyword(week0_posts['Post Text'].iloc[i],keyword_list):
    list0.append(i)
  for j in week0_posts['Total Comments'][i]:
    if has_any_keyword(j,keyword_list):
        list0.append(i)
emptylist0=[*set(list0)]
newdf0=week0_posts.iloc[emptylist0]

In [7]:
newdf0.head()
newdf0['Combined Title and Subtext']=newdf0['Title']+'            '+newdf0['Post Text']
newdf0=newdf0[['Score','ID','Total Comments','Combined Title and Subtext']]

for i in range(0,len(newdf0)):
  #Combined
  if has_any_keyword(newdf0['Combined Title and Subtext'].iloc[i],keyword_list)==False:
    combined = str(newdf0.at[newdf0.index[i],'Combined Title and Subtext'])
    newdf0['Combined Title and Subtext'].replace(combined,"", inplace=True)
  #Comments
  comments = newdf0['Total Comments'].iloc[i]
  for j in comments:
    if has_any_keyword(j,keyword_list) == False:
      newdf0['Total Comments'].iloc[i].remove(j)
        

newdf0.replace('', np.nan, inplace=True)
newdf0

for i in range(0, len(newdf0)):
  if newdf0['Total Comments'].iloc[i]==[]:
    newdf0['Total Comments'].iloc[i]=np.nan

<ipython-input-7-718bc7b2aadc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf0['Combined Title and Subtext']=newdf0['Title']+'            '+newdf0['Post Text']
/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

In [8]:
newdf0

,Score,ID,Total Comments,Combined Title and Subtext
512,71,yxw3cb,[The strikers should surround that building go...,Striking Academic Workers formed a picket a li...
257,344,zeo91z,[I’m just trynna go to class without being blo...,NaN
259,41,zeh3su,"[Just an explanation, landlords are allowed to...",NaN
515,6,yxk98b,[I worked as a legislative aide for a number o...,NaN
261,115,zeghsv,"[Lots 23, 27, and 29 are open, spread the word...",Parking lot been blocked WTF are yo...
...,...,...,...,...
250,0,zf2lc6,"[Or, hear me out, people are genuinely furious...",Conspiracy time The UC system is pa...
501,194,yy6de8,[The title was required for me to put. Idk why...,Strike Ritual
380,50,z77yyi,[The internal vice president hasn't been doing...,NaN
502,417,yy4nhs,[I'm assuming the school gets away with it bec...,Grad worker union bargaining: FAQs for undergr...


In [9]:
commentsdf0=newdf0['Total Comments']
commentsdf0=commentsdf0.explode('Total Comments')
commentsdf0=commentsdf0.dropna()
newdf0['Combined Title and Subtext']=newdf0['Combined Title and Subtext'].dropna()
combined_df0 = pd.concat([commentsdf0, newdf0['Combined Title and Subtext']], ignore_index=True,axis=0)
print(f'The sample size of posts/comments in total is {len(combined_df0)}')


The sample size of posts/comments in total is 1135


In [10]:
combined_df0

0       The strikers should surround that building goi...
1       This is part of a growing global strike wave, ...
2       Lol literally posting a union president campai...
3       The strike that should have started last year?...
4       One could tell from the meeting that just ende...
                              ...                        
1130    Conspiracy time            The UC system is pa...
1131                            Strike Ritual            
1132                                                  NaN
1133    Grad worker union bargaining: FAQs for undergr...
1134    For or against the strike I want to know…     ...
Length: 1135, dtype: object

In [12]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [13]:
sid = SentimentIntensityAnalyzer()

res0 = [*combined_df0.dropna().apply(sid.polarity_scores)]

In [15]:
sentiment_df = pd.DataFrame.from_records(res0)
newerdf0 = pd.concat([combined_df0, sentiment_df], axis=1, join='inner')

In [17]:
THRESHOLD = 0.15

#week 0
conditions = [
    (newerdf0['compound'] <= -THRESHOLD),
    (newerdf0['compound'] > -THRESHOLD) & (newerdf0['compound'] < THRESHOLD),
    (newerdf0['compound'] >= THRESHOLD),
    ]

values = ["negative", "neutral", "positive"]
newerdf0['sentiment'] = np.select(conditions, values)

In [18]:
print("all")
newerdf0.sentiment.value_counts()

all


positive    449
negative    393
neutral     262
Name: sentiment, dtype: int64